In [1]:
from builtins import Exception
import urllib3
from bs4 import BeautifulSoup
from requests import get
import mysql.connector
import re
from datetime import date
from datetime import datetime

In [87]:
class Liputan6:
     #deklarasi variable global
    url = ""
    mydb = ""
    keyword = ""
    urlDetailData = ""
    today = datetime.now()
    
    #fungsi dari inisialisasi data saat memanggil class pertama kali
    def __init__(self, urlBerita, pencarian):
        self.url = urlBerita
        self.keyword = pencarian
        self.mydb = mysql.connector.connect(host="localhost",user="root", password="",database="db-berita-mca")
    
    #fungsi menyimpan data ke database
    def simpanDatabase(self,data):
        mycursor = self.mydb.cursor()
        sql = "INSERT INTO tb_berita (media,judul_berita,label,penulis,release_date,url,content) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        mycursor.execute(sql, data)
        self.mydb.commit()
        return
    
    #fungsi buat validasi data duplicate di database
    def validasiDataDatabase(self,dataBerita):
        mycursor = self.mydb.cursor()
        sql = "SELECT * FROM tb_berita WHERE url = %s"
        dataUrl = dataBerita
        url = (dataBerita,)
        mycursor.execute(sql, url)
        myresult = mycursor.fetchall()
        return myresult
        
    def getUrlDetail(self,page):
        self.urlDetailData = []
        for page in range(page): 
            if(self.keyword == ""):
                dateNow = self.today.strftime("%Y/%m/%d")
                urlGet = (self.url+"/news/indeks/"+dateNow+"?page={}".format(page+1))
                soup = BeautifulSoup(get(urlGet).text, 'lxml')
                getContent = soup.find('article','main')
                groupContent = getContent.find_all('h4','articles--rows--item__title')
                for x in groupContent:
                    news_url = x.find('a', href=True).get('href')
                    mergeUrl = (news_url+"?page=all")
                    self.urlDetailData.append(mergeUrl)
            else:
                dateNow = self.today.strftime("%d/%m/%Y")
                date = dateNow.replace('/','%2F')
                urlGet = (self.url+"/search?order=latest&chammel_id=&from_date="+date+"&to_date="+date+"&type=all&q="+self.keyword)
                soup = BeautifulSoup(get(urlGet).text, 'lxml')
                getContent = soup.find('article','main')
                groupContent = getContent.find_all('h4','articles--iridescent-list--text-item__title')
                for x in groupContent:
                    news_url = x.find('a', href=True).get('href')
                    mergeUrl = (news_url+"?page=all")
                    self.urlDetailData.append(mergeUrl)
                    
                    
    def scrapingDataTribun(self):
        for dataBerita in self.urlDetailData:
            myresult = self.validasiDataDatabase(dataBerita)
            if(len(myresult)==0):
                try:
                    media = "Liputan6.com"
                    web_data = get(dataBerita)
                    soup = BeautifulSoup(web_data.text, 'html.parser')
                    page = soup.find('div','inner-container-article')
                    title = page.find('h1','read-page--header--title entry-title').text
                    labelGroup = page.find_all('a','read-page--breadcrumb--item__title')
                    for x in labelGroup:
                        label = x.find('span').text
                    author = page.find('span','read-page--header--author__name fn').text
                    release_date = page.find('time','read-page--header--author__datetime updated').text      
                    groupContent = page.find('div','read-page--content')
                    texts = groupContent.find_all('p')
                    news_content = ' '.join([text.text for text in texts]            
                    data = (media,title,label,author,release_date,dataBerita,news_content)
                    self.simpanDatabase(data)
                except Exception as e:
                    continue 
            else:
                continue

SyntaxError: invalid syntax (<ipython-input-87-5dfc13595611>, line 77)

In [88]:
url = 'https://www.liputan6.com'
pencarian = ''
testing = Liputan6(url, pencarian)
testing.getUrlDetail(1)
testing.scrapingDataTribun()

https://www.liputan6.com/news/read/4789573/ridwan-kamil-siapkan-jurus-cegah-penyebaran-covid-19-varian-omicron-di-jawa-barat?page=all
Liputan6.com, Jakarta - Gubernur Jawa Barat Ridwan Kamil mengaku telah menyiapkan sejumlah jurus untuk mencegah penularan Covid-19 varian Omicron yang sudah masuk ke Indonesia. Strategi yang disiapkan menghadapi varian Omicron, salah satunya dengan menyiapkan instrumen penanganan kesehatan, sama seperti yang dilakukan Jabar saat menangani varian Delta beberapa waktu lalu. Baca Juga "Dulu Delta terdeteksi di Karawang, menyebar dengan cepat. Oleh karena itu kami belajar dari delta. Per hari ini Jabar sedang menyiapkan semua perlengkapan seperti menghadapi virus Delta," kata Ridwan Kamil, di Bogor, seperti dilansir Antara, Sabtu 18 Desember 2021. Selain itu, Jabar akan memperkuat stok oksigen. Terkait hal ini, dia mendapatkan dukungan dari komunitas Indonesia Pasti Bisa. Juga menggenjot telusur, tes dan tindak lanjut (3T). "Jadi tabung oksigen kita cek lagi

In [85]:
today = datetime.now()
dateNow = today.strftime("%d/%m/%Y")
print(dateNow.replace('/','%2F'))
      

19%2F12%2F2021
